<a href="https://colab.research.google.com/github/sikoh/DS-Linear-Models/blob/main/Logistic-Regression/DS_LogisticRegression_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module Project: Logistic Regression

Do you like burritos? 🌯 You're in luck then, because in this project you'll create a model to predict whether a burrito is `'Great'`.

The dataset for this assignment comes from [Scott Cole](https://srcole.github.io/100burritos/), a San Diego-based data scientist and burrito enthusiast.

## Directions

The tasks for this project are the following:

- **Task 1:** Import `csv` file using `wrangle` function.
- **Task 2:** Conduct exploratory data analysis (EDA), and modify `wrangle` function .
- **Task 3:** Split data into feature matrix `X` and target vector `y`.
- **Task 4:** Split feature matrix `X` and target vector `y` into training and test sets.
- **Task 5:** Establish the baseline accuracy score for your dataset.
- **Task 6:** Build `model_logr` using a pipeline that includes three transfomers and `LogisticRegression` predictor. Train model on `X_train` and `X_test`.
- **Task 7:** Calculate the training and test accuracy score for your model.
- **Task 8:** Create a horizontal bar chart showing the 10 most influencial features for your  model.
- **Task 9:** Demonstrate and explain the differences between `model_lr.predict()` and `model_lr.predict_proba()`.

**Note**

You should limit yourself to the following libraries:

- `category_encoders`
- `matplotlib`
- `pandas`
- `sklearn`

# I. Wrangle Data

In [ ]:
import pandas as pd


In [ ]:
def wrangle(filepath):
    # Import w/ DateTimeIndex
    df = pd.read_csv(filepath, parse_dates=['Date'],
                     index_col='Date')

    # Drop unrated burritos
    df.dropna(subset=['overall'], inplace=True)

    # Derive binary classification target:
    # We define a 'Great' burrito as having an
    # overall rating of 4 or higher, on a 5 point scale
    df['Great'] = (df['overall'] >= 4).astype(int)

    # Drop high cardinality categoricals
    df = df.drop(columns=['Notes', 'Location', 'Address', 'URL', 'Neighborhood'])

    # Drop columns to prevent "leakage"
    df = df.drop(columns=['Rec', 'overall'])

    return df


**Task 1:** Use the above `wrangle` function to import the `burritos.csv` file into a DataFrame named `df`.

In [ ]:
filepath = "https://raw.githubusercontent.com/bloominstituteoftechnology/DS-Unit-2-Linear-Models/master/data/burritos/burritos.csv"
df = wrangle(filepath)
df

Burrito  Yelp  Google Chips  Cost  Hunger  Mass (g)  \
Date                                                                    
2016-01-18    California    3.5     4.2   NaN  6.49     3.0       NaN   
2016-01-24    California    3.5     3.3   NaN  5.45     3.5       NaN   
2016-01-24       Carnitas   NaN     NaN   NaN  4.85     1.5       NaN   
2016-01-24    Carne asada   NaN     NaN   NaN  5.25     2.0       NaN   
2016-01-27     California   4.0     3.8     x  6.59     4.0       NaN   
...                   ...   ...     ...   ...   ...     ...       ...   
2019-08-27      Al Pastor   NaN     NaN   NaN  6.00     1.0       NaN   
2019-08-27  Chile Relleno   NaN     NaN   NaN  6.00     4.0       NaN   
2019-08-27     California   NaN     NaN   NaN  7.90     3.0       NaN   
2019-08-27         Shrimp   NaN     NaN   NaN  7.90     3.0       NaN   
2019-08-27    Pollo Asado   NaN     NaN   NaN  5.50     3.5       NaN   

            Density (g/mL)  Length  Circum  ...  Lobster  Queso  Egg  \
Date                                        ...                        
2016-01-18             NaN     NaN     NaN  ...      NaN    NaN  NaN   
2016-01-24             NaN     NaN     NaN  ...      NaN    NaN  NaN   
2016-01-24             NaN     NaN     NaN  ...      NaN    NaN  NaN   
2016-01-24             NaN     NaN     NaN  ...      NaN    NaN  NaN   
2016-01-27             NaN     NaN     NaN  ...      NaN    NaN  NaN   
...                    ...     ...     ...  ...      ...    ...  ...   
2019-08-27             NaN    17.0    20.5  ...      NaN    NaN  NaN   
2019-08-27             NaN    19.0    26.0  ...      NaN    NaN  NaN   
2019-08-27             NaN    20.0    22.0  ...      NaN    NaN  NaN   
2019-08-27             NaN    22.5    24.5  ...      NaN    NaN  NaN   
2019-08-27             NaN    17.0    21.3  ...      NaN    NaN  NaN   

            Mushroom  Bacon  Sushi  Avocado  Corn  Zucchini  Great  
Date                                                                
2016-01-18       NaN    NaN    NaN      NaN   NaN       NaN      0  
2016-01-24       NaN    NaN    NaN      NaN   NaN       NaN      0  
2016-01-24       NaN    NaN    NaN      NaN   NaN       NaN      0  
2016-01-24       NaN    NaN    NaN      NaN   NaN       NaN      0  
2016-01-27       NaN    NaN    NaN      NaN   NaN       NaN      1  
...              ...    ...    ...      ...   ...       ...    ...  
2019-08-27       NaN    NaN    NaN      NaN   NaN       NaN      0  
2019-08-27       NaN    NaN    NaN      NaN   NaN       NaN      1  
2019-08-27       NaN    NaN    NaN      NaN   NaN       NaN      0  
2019-08-27       NaN    NaN    NaN      NaN   NaN       NaN      1  
2019-08-27       NaN    NaN    NaN      NaN   NaN       NaN      1  

[421 rows x 59 columns]

During your exploratory data analysis, note that there are several columns whose data type is `object` but that seem to be a binary encoding. For example, `df['Beef'].head()` returns:

```
0      x
1      x
2    NaN
3      x
4      x
Name: Beef, dtype: object
```

**Task 2:** Change the `wrangle` function so that these columns are properly encoded as `0` and `1`s. Be sure your code handles upper- and lowercase `X`s, and `NaN`s.

In [ ]:
# Conduct your exploratory data analysis here
# And modify the `wrangle` function above.

If you explore the `'Burrito'` column of `df`, you'll notice that it's a high-cardinality categorical feature. You'll also notice that there's a lot of overlap between the categories.

**Stretch Goal:** Change the `wrangle` function above so that it engineers four new features: `'california'`, `'asada'`, `'surf'`, and `'carnitas'`. Each row should have a `1` or `0` based on the text information in the `'Burrito'` column. For example, here's how the first 5 rows of the dataset would look.

| **Burrito** | **california** | **asada** | **surf** | **carnitas** |
| :---------- | :------------: | :-------: | :------: | :----------: |
| California  |       1        |     0     |    0     |      0       |
| California  |       1        |     0     |    0     |      0       |
|  Carnitas   |       0        |     0     |    0     |      1       |
| Carne asada |       0        |     1     |    0     |      0       |
| California  |       1        |     0     |    0     |      0       |

**Note:** Be sure to also drop the `'Burrito'` once you've engineered your new features.

In [ ]:
# Conduct your exploratory data analysis here
# And modify the `wrangle` function above.

# II. Split Data

**Task 3:** Split your dataset into the feature matrix `X` and the target vector `y`. You want to predict `'Great'`.

In [ ]:
X = ...
y = ...

**Task 4:** Split `X` and `y` into a training set (`X_train`, `y_train`) and a test set (`X_test`, `y_test`).

- Your training set should include data from 2016 through 2017.
- Your test set should include data from 2018 and later.

In [ ]:
X_train, y_train = ..., ...
X_test, y_test = ..., ...

# III. Establish Baseline

**Task 5:** Since this is a **classification** problem, you should establish a baseline accuracy score. Figure out what is the majority class in `y_train` and what percentage of your training observations it represents.

In [ ]:
baseline_acc = ...
print('Baseline Accuracy Score:', baseline_acc)

# IV. Build Model

**Task 6:** Build a `Pipeline` named `model_logr`, and fit it to your training data. Your pipeline should include:

- a `OneHotEncoder` transformer for categorical features,
- a `SimpleImputer` transformer to deal with missing values,
- a [`StandarScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) transfomer (which often improves performance in a logistic regression model), and
- a `LogisticRegression` predictor.

In [ ]:
model_logr = ...

# IV. Check Metrics

**Task 7:** Calculate the training and test accuracy score for `model_logr`.

In [ ]:
training_acc = ...
test_acc = ...

print('Training MAE:', training_acc)
print('Test MAE:', test_acc)

# V. Communicate Results

**Task 8:** Create a horizontal barchart that plots the 10 most important coefficients for `model_lr`, sorted by absolute value.

**Note:** Since you created your model using a `Pipeline`, you'll need to use the [`named_steps`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) attribute to access the coefficients in your `LogisticRegression` predictor. Be sure to look at the shape of the coefficients array before you combine it with the feature names.

In [ ]:
# Create your horizontal barchart here.

There is more than one way to generate predictions with `model_lr`. For instance, you can use [`predict`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logisticregression) or [`predict_proba`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logisticregression#sklearn.linear_model.LogisticRegression.predict_proba).

**Task 9:** Generate predictions for `X_test` using both `predict` and `predict_proba`. Then below, write a summary of the differences in the output for these two methods. You should answer the following questions:

- What data type do `predict` and `predict_proba` output?
- What are the shapes of their different output?
- What numerical values are in the output?
- What do those numerical values represent?

In [ ]:
# Write code here to explore the differences between `predict` and `predict_proba`.

**Give your written answer here:**

```


```
